In [1]:
import cv2
import imutils
import dlib
import scipy
import numpy as np

from scipy.spatial import distance
from imutils import face_utils

import pygame

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def light_remover(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    lab =  cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    L = lab[:,:,0]
    composed = cv2.cvtColor(lab,cv2.COLOR_LAB2BGR)
    # invert the image lighting
    Medianblur = cv2.medianBlur(L,99)
    invertedL = cv2.bitwise_not(Medianblur)
    composed = cv2.addWeighted(gray,0.80,invertedL,0.20,0)
    return composed

In [3]:
def eye_aspect_ratio(eye):
	A = distance.euclidean(eye[1], eye[5])
	B = distance.euclidean(eye[2], eye[4])
	C = distance.euclidean(eye[0], eye[3])
	ear = (A + B) / (2.0 * C)
# 	print(ear)
	return ear

In [4]:
def mouth_aspect_ratio(muh):
    A = distance.euclidean(muh[13], muh[19])
    B = distance.euclidean(muh[15], muh[17])
    C = distance.euclidean(muh[12], muh[16])
    ear1 = (A + B) / (2.0 * C)
#     print(ear1)
    return ear1

In [5]:
def select_alarm() :
    sound_alarm("nomal_alarm.wav")


def sound_alarm(path) :
    pygame.mixer.init()
    pygame.mixer.music.load(path)
    pygame.mixer.music.play()

In [6]:
threshEye = 0.32
threshLip = 0.28
frame_check = 60
frame_check1 = 120
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(uStart, uEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

## Via Mobile

In [ ]:
url = 'http://192.168.43.1:8080/video'

cap = cv2.VideoCapture(url)
flag=0
global lipear
global ear
while True:
    ret, frame=cap.read()
    frame  = light_remover(frame)
    frame = imutils.resize(frame, width=450)
    subjects = detect(frame, 0)
    for subject in subjects:
        shape = predict(frame, subject)
        shape = face_utils.shape_to_np(shape)#converting to NumPy Array
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        lips = shape[uStart:uEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        lipear = mouth_aspect_ratio(lips)
        ear = (leftEAR + rightEAR) / 2.0
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        lipHull = cv2.convexHull(lips)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [lipHull], -1, (0, 255, 0), 1)
        print(lipear)
        if ear < threshEye or lipear > threshLip:
            flag += 1
            #print (flag)
            if flag >= frame_check:
                if lipear > threshLip and ear < threshEye:
                    cv2.putText(frame, "Yawn", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    cv2.putText(frame, "Eyes", (10,325),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                elif ear < threshEye: 
                    select_alarm()
                    cv2.putText(frame, "Eyes", (10,325),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                elif lipear > threshLip:
                    cv2.putText(frame, "Yawn", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    
        else:
            flag = 0
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        cv2.destroyAllWindows()
        cap.release()
        break



# Via Laptop

In [7]:

cap = cv2.VideoCapture(0)
flag=0
global lipear
global ear
while True:
    ret, frame=cap.read()
    frame  = light_remover(frame)
    frame = imutils.resize(frame, width=450)
    subjects = detect(frame, 0)
    for subject in subjects:
        shape = predict(frame, subject)
        shape = face_utils.shape_to_np(shape)#converting to NumPy Array
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        lips = shape[uStart:uEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        lipear = mouth_aspect_ratio(lips)
        ear = (leftEAR + rightEAR) / 2.0
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        lipHull = cv2.convexHull(lips)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [lipHull], -1, (0, 255, 0), 1)
        print(lipear)
        if ear < threshEye or lipear > threshLip:
            flag += 1
            #print (flag)
            if flag >= frame_check:
                if lipear > threshLip and ear < threshEye:
                    cv2.putText(frame, "Yawn", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    cv2.putText(frame, "Eyes", (10,325),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                elif ear < threshEye: 
                    select_alarm()
                    cv2.putText(frame, "Eyes", (10,325),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                elif lipear > threshLip:
                    cv2.putText(frame, "Yawn", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    
        else:
            flag = 0
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        cv2.destroyAllWindows()
        cap.release()
        break


0.04993761694389223
0.04993761694389223
0.07669649888473705
0.043643037779800366
0.025565499628245683
0.05574513610066167
0.05574513610066167
0.06729033180527572
0.051130999256491366
0.09394817552057834
0.06911873322463923
0.15445093048825997
0.11523319193960638
0.17145425398121888
0.11631048999607828
0.11523319193960638
0.051130999256491366
0.08083109461724323
0.10493664722668089
0.07669649888473705
0.062422021179865284
0.11504474832710557
0.07669649888473705
0.07478994824163422
0.049859965494422814
0.10913579542041725
0.10359890641335218
0.10790660599072865
0.05543148292854333
0.10629569525438572
0.09404661260758033
0.05402515123989041
0.0765216491239271
0.056912549151328436
0.08083109461724323
0.08279161998374318
0.08279161998374318
0.11495371030657049
0.05866136913244332
0.12588419111880975
0.10493664722668089
0.08083109461724323
0.15409881913186899
0.08908415798097151
0.11504474832710557
0.10226199851298273
0.0643477465021563
0.10226199851298273
0.10359890641335218
0.1290932398102